# Points of Dispense - Template for Minimizing Maximum Distance
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### Optimization

Read in data from ...

In [2]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')

In [3]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [4]:
dist = dist_miles.values
N = population['population'].values
prop = population['global'].values

Create indices for block groups and PODs

In [5]:
blocks = range(len(N))
pods = range(len(dist[0]))

Initialize model for POD locations

In [6]:
m = gp.Model('POD_locations')

Using license file C:\Users\Elizabeth\gurobi.lic
Academic license - for non-commercial use only


Add decision variables x[i] for whether a POD is opened and y[i,j] for whether POD i serves block group j

In [7]:
x = m.addVars(pods, vtype=GRB.BINARY, name='x')
y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='y')
w = m.addVars(blocks, vtype=GRB.BINARY, name='w')

Add auxiliary variable for maximum distance

In [8]:
z = m.addVar(vtype=GRB.CONTINUOUS, name='z')

Set up objective function to minimize total distance across the population

In [9]:
m.setObjective(z, GRB.MINIMIZE)

Constraint: y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened

In [10]:
m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

Constraint: each block group must be assigned one shelter

In [11]:
m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

Constraint: number of PODs opened must be 47

In [12]:
m.addConstr((gp.quicksum(x[i] for i in pods) <= 47), name='pods_opened')

<gurobi.Constr *Awaiting Model Update*>

Constraint: w[j] should be 1 if prop[j] is not zero, and zero otherwise

In [ ]:
m.addConstrs((w[j] * 9999 >= prop[j] * N[j] for j in blocks), 
             name='Prop_Above_Zero')

Constraint: maximum distance

In [14]:
m.addConstrs((z >= dist[j, i] * w[j] * y[j, i] for i in pods for j in blocks), 
             name='Distance_Below_Max')

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (0, 6): <gurobi.Constr *Awaiting Model Update*>,
 (0, 7): <gurobi.Constr *Awaiting Model Update*>,
 (0, 8): <gurobi.Constr *Awaiting Model Update*>,
 (0, 9): <gurobi.Constr *Awaiting Model Update*>,
 (0, 10): <gurobi.Constr *Awaiting Model Update*>,
 (0, 11): <gurobi.Constr *Awaiting Model Update*>,
 (0, 12): <gurobi.Constr *Awaiting Model Update*>,
 (0, 13): <gurobi.Constr *Awaiting Model Update*>,
 (0, 14): <gurobi.Constr *Awaiting Model Update*>,
 (0, 15): <gurobi.Constr *Awaiting Model Update*>,
 (0, 16): <gurobi.Constr *Awaiting Model Update*>,
 (0, 17): <gurobi.Constr *Awaiting Model Update*>,
 (0, 18): <gurobi.Constr *Awaiting Model Update*>,
 (0, 19): <gurobi.Constr *Awaiting Model 

Optimize model

In [15]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 104501 rows, 51748 columns and 258547 nonzeros
Model fingerprint: 0x1f644705
Variable types: 1 continuous, 51747 integer (51747 binary)
Coefficient statistics:
  Matrix range     [8e-02, 4e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve time: 0.50s
Presolved: 104501 rows, 51748 columns, 258547 nonzeros
Variable types: 1 continuous, 51747 integer (51747 binary)
Found heuristic solution: objective 36.3382164
Found heuristic solution: objective 30.4291814
Found heuristic solution: objective 30.1723360

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Root relaxation: objective 7.133241e-01, 18746 iterations, 2.59 seconds
Total elapsed time = 7.96s

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntIn

Which block groups were assigned to which shelters?

In [16]:
block_pod_list = [[j, i] for i in pods for j in blocks if y[j, i].x==1]